<a href="https://colab.research.google.com/github/lefkothea1/NLP_2/blob/main/bonusassignment_Milena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#install at every runtime?-prob yes
!pip install checklist
!pip install torch

#should i install it every time?-yes
!pip install simpletransformers 

  Attempting uninstall: jupyter-console
    Found existing installation: jupyter-console 5.2.0
    Uninstalling jupyter-console-5.2.0:
      Successfully uninstalled jupyter-console-5.2.0


In [34]:
import checklist
import torch
import spacy
import numpy as np
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#$loading eng
nlp = spacy.load("en_core_web_sm")

In [35]:
# from platform import python_version #to check version
# python_version()

import pandas as pd
import checklist
from checklist.perturb import Perturb
#from checklist.perturb import add_negation
import nltk #; nltk.download('omw-1.4')
#nltk.download()

In [37]:
#setting up google drive and loading diagnostic test data
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data = pd.read_csv("/content/drive/MyDrive/NLP2/data/Assignment 2_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv")
url = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv'
#data = pd.read_csv(url)

#$loading eng
#nlp = spacy.load("en_core_web_sm")
print(data)

Mounted at /content/drive
       id                                               text  labels
0   89200      @USER @USER Who the hell does he think he is?       1
1   71294  #BREAKING. #Greece: Molotov cocktails fly afte...       1
2   55633  #OrrinHatch I can’t believe this sexist , clue...       1
3   16856  @USER @USER I'll use that one the next time im...       1
4   26072    0-1 lost my acca on the first fucking fight cba       1
5   22067  #Bakersfield is why we need gun control! Sorry...       1
6   60466  #Christian #America – If we go by #Trump’s exa...       1
7   25685  @USER @USER @USER She is the most disingenuous...       1
8   16333  #Democrats #Liberals you are being #threatened...       1
9   78417  699. Just wanted to tell you you should Hang O...       1
10  76379  #LiberalHypocrisy #criminal #democrat #evidenc...       1
11  21826                (cr1tikal voice) smef my ass cheeks       1
12  15998                             *gets all the bitches*       1
13  9645

In [38]:
#loading finetuned MODEL FROM DRIVE
bert_mod = ClassificationModel("bert", "/content/drive/MyDrive/NLP2/trained_models3")

In [39]:
# Diagnostic test 1
# expanding contractions
#adding typos from tutorial
from checklist.perturb import Perturb

pdata = list(data['text'])
labels = list(data['labels'])
ret = Perturb.perturb(pdata, Perturb.expand_contractions)
ret

MunchWithAdd({'data': [['@USER @USER Who the hell does he think he is?', '@USER @USER Who the hell does he think he is?'], ['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL', '#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL'], ['#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!', '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!'], ["@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes.

In [40]:
#keeping only the expansions and recreating them into a df
exp_lst=[]
for i in range(len(ret.data)):
  exp_lst.append(ret.data[i][1])

exp_df = pd.DataFrame(list(zip(exp_lst, labels)), columns =['text', 'label'])
print(exp_df)

                                                 text  label
0       @USER @USER Who the hell does he think he is?      1
1   #BREAKING. #Greece: Molotov cocktails fly afte...      1
2   #OrrinHatch I can’t believe this sexist , clue...      1
3   @USER @USER I will use that one the next time ...      1
4     0-1 lost my acca on the first fucking fight cba      1
5   #Bakersfield is why we need gun control! Sorry...      1
6   #Christian #America – If we go by #Trump’s exa...      1
7   @USER @USER @USER She is the most disingenuous...      1
8   #Democrats #Liberals you are being #threatened...      1
9   699. Just wanted to tell you you should Hang O...      1
10  #LiberalHypocrisy #criminal #democrat #evidenc...      1
11                (cr1tikal voice) smef my ass cheeks      1
12                             *gets all the bitches*      1
13  #Antifa: Take note of how to protest with civi...      1
14  @USER omg is he for real ?!?!!??????) this hap...      1
15  #Conservatism101   I

In [41]:
#geting predictions on the expansions
prediction_exp, out_nr_typo = bert_mod.predict(exp_lst)
prediction_reg, out_nr_reg = bert_mod.predict(pdata)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [42]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
result, model_outputs, wrong_predictions = bert_mod.eval_model(data, clf_report = classification_report)
result_exp, model_outputs_exp, wrong_predictions_exp = bert_mod.eval_model(exp_df, clf_report = classification_report)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [21]:
print(result['clf_report'])
print(result_exp['clf_report'])

print(wrong_predictions_exp)

              precision    recall  f1-score   support

         0.0       0.78      0.90      0.83        50
         1.0       0.88      0.74      0.80        50

    accuracy                           0.82       100
   macro avg       0.83      0.82      0.82       100
weighted avg       0.83      0.82      0.82       100

              precision    recall  f1-score   support

         0.0       0.77      0.88      0.82        50
         1.0       0.86      0.74      0.80        50

    accuracy                           0.81       100
   macro avg       0.82      0.81      0.81       100
weighted avg       0.82      0.81      0.81       100

[{'guid': 1, 'text_a': '#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL', 'text_b': None, 'label': 1}, {'guid': 3, 'text_a': "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can choose to be irresponsible or choose

In [99]:
# show whole df
pd.set_option('display.max_rows', 1000)

summary = pd.DataFrame(list(zip(exp_lst, prediction_exp, prediction_reg, labels)), columns = ['text', 'exp', 'original', 'label'])
print(summary)

print(summary.loc[93][0])

                                                 text  exp  original  label
0       @USER @USER Who the hell does he think he is?    1         1      1
1   #BREAKING. #Greece: Molotov cocktails fly afte...    0         0      1
2   #OrrinHatch I can’t believe this sexist , clue...    1         1      1
3   @USER @USER I will use that one the next time ...    0         0      1
4     0-1 lost my acca on the first fucking fight cba    1         1      1
5   #Bakersfield is why we need gun control! Sorry...    1         1      1
6   #Christian #America – If we go by #Trump’s exa...    0         0      1
7   @USER @USER @USER She is the most disingenuous...    1         1      1
8   #Democrats #Liberals you are being #threatened...    0         0      1
9   699. Just wanted to tell you you should Hang O...    1         1      1
10  #LiberalHypocrisy #criminal #democrat #evidenc...    1         1      1
11                (cr1tikal voice) smef my ass cheeks    1         1      1
12          

In [82]:
# Diagnostic test 2
# removing punctuation to prevent censoring using punctuation marks
import string

def strip_punctuation2(line):
  """Removes punctuation
  Parameters
  ----------
  inputs string
  Returns
  -------
  string
  With punctuation stripped
  """
  # doc is a spacy doc
  newline = str()
  for char in line:
   if char not in string.punctuation:
       newline = newline + char
  return newline

ret = []
perturbed = []

pdata = list(data['text'])
labels = list(data['labels'])
for line in pdata:
  newline = strip_punctuation2(line)
  set = [line, newline]
  ret.append(set)
  perturbed.append(newline)
#ret

In [83]:
strip_df = pd.DataFrame(list(zip(perturbed, labels)), columns =['text', 'label'])
#print(strip_df)

In [84]:
#geting predictions on the expansions
prediction_strip, out_nr_strip = bert_mod.predict(perturbed)
prediction_reg, out_nr_reg = bert_mod.predict(pdata)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [86]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
result, model_outputs, wrong_predictions = bert_mod.eval_model(data, clf_report = classification_report)
result_strip, model_outputs_strip, wrong_predictions_strip = bert_mod.eval_model(strip_df, clf_report = classification_report)

print(result['clf_report'])
print(result_strip['clf_report'])

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         0.0       0.78      0.90      0.83        50
         1.0       0.88      0.74      0.80        50

    accuracy                           0.82       100
   macro avg       0.83      0.82      0.82       100
weighted avg       0.83      0.82      0.82       100

              precision    recall  f1-score   support

         0.0       0.80      0.94      0.86        50
         1.0       0.93      0.76      0.84        50

    accuracy                           0.85       100
   macro avg       0.86      0.85      0.85       100
weighted avg       0.86      0.85      0.85       100



In [98]:
# show whole df
pd.set_option('display.max_rows', 1000)

summary = pd.DataFrame(list(zip(perturbed, prediction_strip, prediction_reg, labels)), columns = ['text', 'typo', 'original', 'label'])
print(summary)

print(summary.loc[88][0])

                                                 text  typo  original  label
0          USER USER Who the hell does he think he is     1         1      1
1   BREAKING Greece Molotov cocktails fly after pr...     0         0      1
2   OrrinHatch I can’t believe this sexist  cluele...     1         1      1
3   USER USER Ill use that one the next time im in...     1         0      1
4      01 lost my acca on the first fucking fight cba     1         1      1
5   Bakersfield is why we need gun control Sorry f...     1         1      1
6   Christian America – If we go by Trump’s exampl...     0         0      1
7   USER USER USER She is the most disingenuous pe...     1         1      1
8   Democrats Liberals you are being threatened by...     1         0      1
9   699 Just wanted to tell you you should Hang Ou...     1         1      1
10  LiberalHypocrisy criminal democrat evidence el...     1         1      1
11                  cr1tikal voice smef my ass cheeks     1         1      1